In [1]:
from models import *
from models.pointnet2_cls_msg import *
import torch

In [12]:
import sys
import os  # Import os to easily manipulate paths

# This should be the directory that contains your module, not the module file itself
package_directory = os.path.dirname('/home/hani/GitHub/PCR_practice/Old_notebooks')

# Add the directory to sys.path
if package_directory not in sys.path:
    sys.path.append(package_directory)

# Now you can import your module as if it were in the same directory
from Old_notebooks.simdata_utils import *
from Old_notebooks import *


In [3]:
checkpoint_path_pnpp_original  = 'log/classification/pointnet2_msg_normals/checkpoints/best_model.pth'
checkpoint_path_pnpp_mdnetc = '/home/hani/GitHub/PCR_practice/Old_notebooks/models/mdnet_40_C_pnpp_da/model_best_test.pth'

In [4]:
checkpoint_pnpp_original = torch.load(checkpoint_path_pnpp_original, map_location=torch.device('cpu'))
checkpoint_pnpp_mdnetc = torch.load(checkpoint_path_pnpp_mdnetc, map_location=torch.device('cpu'))

In [5]:
checkpoint_pnpp_original.keys()

dict_keys(['epoch', 'instance_acc', 'class_acc', 'model_state_dict', 'optimizer_state_dict'])

In [6]:
checkpoint_pnpp_mdnetc.keys()

dict_keys(['cfg', 'epoch', 'model_state', 'optimizer_state', 'lr_sched_state', 'bnm_sched_state', 'test_perf'])

In [7]:
### load the state dict of the model

model_state_dict_pnpp_original = checkpoint_pnpp_original['model_state_dict']
model_state_dict_pnpp_mdnetc = checkpoint_pnpp_mdnetc['model_state']


In [23]:
### Load architecture of the model original
model_pnpp_original  = get_model(40,normal_channel=True)
model_pnpp_original.load_state_dict(model_state_dict_pnpp_original)

<All keys matched successfully>

In [147]:
### set the model to evaluation mode
#model_pnpp_original.eval()

In [25]:
### loading data
# Example usage
#hdf5_file = 'partial_mdnet_40.h5'
hdf5_file = '/home/hani/GitHub/PCR_practice/Old_notebooks/mdnet_sorted_no_corr_clean.h5'
#hdf5_file = 'combined.h5'
# Reading the data
data = load_entire_dataset(hdf5_file)

print('Instances', len(list(data.keys())))

Instances 1300


In [17]:
### extract source and target point clouds ###
src_dst_data =  extract_src_dst(data)
part_dst_data = extract_partViews_dst(data)

In [148]:
#part_dst_data

In [189]:
part_source = part_dst_data['airplane_O10_V1']
part_dst = part_dst_data['airplane_O15_Vdst'] 
part_dst = pc_normalize(part_dst)

In [ ]:
#### data is loaded and ready to be used

In [198]:
additional_features = estimate_robust_normals_using_open3d(smaple_data_with_normals_ds[:,:3])  # Replace with actual features if available
#additional_features = np.random.rand(part_dst.shape[0], 3)  # Replace with actual features if available


In [199]:
augmented_point_cloud_np = np.concatenate([(smaple_data_with_normals_ds[:,:3]),additional_features], axis=1)

In [200]:
augmented_point_cloud_np.shape

(2500, 6)

In [201]:
point_cloud_tensor = torch.tensor(augmented_point_cloud_np, dtype=torch.float).transpose(0, 1).unsqueeze(0)

In [202]:
point_cloud_tensor.shape

torch.Size([1, 6, 2500])

In [203]:
# Make a prediction
with torch.no_grad():
    prediction, l3_pts = model_pnpp_original.forward(point_cloud_tensor)

In [204]:
prediction

tensor([[-7.1526e-07, -1.7786e+01, -1.8443e+01, -1.8961e+01, -2.5635e+01,
         -2.5560e+01, -1.7568e+01, -1.6996e+01, -2.3804e+01, -1.9274e+01,
         -2.2335e+01, -1.8848e+01, -1.9226e+01, -2.2030e+01, -1.9227e+01,
         -2.1648e+01, -2.0232e+01, -1.7183e+01, -2.0889e+01, -2.1294e+01,
         -1.8363e+01, -1.6360e+01, -1.6795e+01, -1.9659e+01, -1.8857e+01,
         -1.9180e+01, -1.6710e+01, -1.6694e+01, -1.9954e+01, -1.7378e+01,
         -1.7797e+01, -2.0409e+01, -2.3182e+01, -1.7018e+01, -1.6416e+01,
         -1.6925e+01, -2.2164e+01, -2.7410e+01, -2.5723e+01, -1.7804e+01]])

In [205]:
_, predicted_class = torch.max(prediction, 1)  # Returns the value and index of the maximum value in each row
# or
predicted_class = prediction.argmax(1)  # Directly returns the index of the maximum value in each row

print(predicted_class)


tensor([0])


In [146]:
l3_pts.shape

torch.Size([1, 1024, 1])

In [119]:
point_cloud_tensor.shape

torch.Size([1, 6, 2500])

In [122]:
viz1pcl_xyz(part_dst)

In [ ]:
##################### Load new dataset resampled with normals

In [149]:
def read_data_to_numpy(filepath):
    """
    Reads a text file with point cloud data into a numpy array.

    Parameters:
    - filepath: str, the path to the text file containing the data.

    Returns:
    - data: numpy.ndarray, an array of shape (n, 6) where n is the number of points.
    """
    # Initialize an empty list to store the data points
    data_points = []

    # Open the file and read line by line
    with open(filepath, 'r') as file:
        for line in file:
            # Split each line into components, convert to float, and append to the list
            point = [float(value) for value in line.strip().split(',')]
            data_points.append(point)

    # Convert the list of data points into a numpy array
    data = np.array(data_points)

    return data

In [165]:
sample_data_path = '/home/hani/GitHub/PCR_practice/modelnet40_normal_resampled/airplane/airplane_0025.txt'

In [166]:
smaple_data_with_normals = read_data_to_numpy(sample_data_path)

In [167]:
smaple_data_with_normals_ds = downsample_data_ndarray(smaple_data_with_normals,100,25)

In [176]:
viz1pcl_xyz(smaple_data_with_normals_ds[:,3:])

In [169]:
viz1pcl_xyz(estimate_robust_normals_using_open3d(smaple_data_with_normals_ds[:,:3]))

In [172]:
viz1pcl_xyz(project_to_unit_sphere(smaple_data_with_normals_ds[:,:3])[0])

In [173]:
def pc_normalize(pc):
    centroid = np.mean(pc, axis=0)
    pc = pc - centroid
    m = np.max(np.sqrt(np.sum(pc**2, axis=1)))
    pc = pc / m
    return pc

In [175]:
viz1pcl_xyz(pc_normalize(smaple_data_with_normals_ds[:,:3]))

In [178]:
viz1pcl_xyz(smaple_data_with_normals_ds[:,:3])